In [2]:
import os
import pandas as pd
from itertools import chain
from SPARQLWrapper import SPARQLWrapper, JSON


In [4]:
persons = pd.read_json(path_or_buf=os.path.join("..", "..", "data/tmp", "persons_Q800.jsonl"), lines=True)

In [5]:
persons

,type,id,labels,descriptions,aliases,claims,sitelinks,pageid,ns,title,lastrevid,modified
0,item,Q57321,"{'lb': {'language': 'lb', 'value': 'Laura Chin...","{'it': {'language': 'it', 'value': 'politica c...","{'ko': [{'language': 'ko', 'value': '로라 친칠라'},...","{'P27': [{'mainsnak': {'snaktype': 'value', 'p...","{'plwikiquote': {'site': 'plwikiquote', 'title...",60140,0,Q57321,2265302423,2024-10-26 12:02:13+00:00
1,item,Q211061,"{'ru': {'language': 'ru', 'value': 'Драмонд, К...","{'fa': {'language': 'fa', 'value': 'بوکسور کاس...","{'ru': [{'language': 'ru', 'value': 'Карл Дэви...","{'P31': [{'mainsnak': {'snaktype': 'value', 'p...","{'ruwiki': {'site': 'ruwiki', 'title': 'Драмон...",206779,0,Q211061,1967460768,2023-09-03 15:55:25+00:00
2,item,Q348649,"{'zh-hans': {'language': 'zh-hans', 'value': '...","{'it': {'language': 'it', 'value': 'attore e b...","{'pl': [{'language': 'pl', 'value': 'Harry Shu...","{'P21': [{'mainsnak': {'snaktype': 'value', 'p...","{'enwiki': {'site': 'enwiki', 'title': 'Harry ...",333338,0,Q348649,2252251487,2024-09-24 03:33:48+00:00
3,item,Q360433,"{'pt': {'language': 'pt', 'value': 'Franklin C...","{'it': {'language': 'it', 'value': 'astronauta...","{'en': [{'language': 'en', 'value': 'Franklin ...","{'P21': [{'mainsnak': {'snaktype': 'value', 'p...","{'commonswiki': {'site': 'commonswiki', 'title...",343572,0,Q360433,2273344752,2024-11-11 18:14:37+00:00
4,item,Q365359,"{'pt': {'language': 'pt', 'value': 'Mario Echa...","{'de': {'language': 'de', 'value': 'costa-rica...","{'fr': [{'language': 'fr', 'value': 'Mario Ech...","{'P21': [{'mainsnak': {'snaktype': 'value', 'p...","{'commonswiki': {'site': 'commonswiki', 'title...",347922,0,Q365359,2285766915,2024-12-10 19:16:31+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
2560,item,Q125573387,"{'en': {'language': 'en', 'value': 'Carlos Ova...","{'en': {'language': 'en', 'value': 'costarican...","{'en': [{'language': 'en', 'value': 'macho'}],...","{'P31': [{'mainsnak': {'snaktype': 'value', 'p...","{'eswiki': {'site': 'eswiki', 'title': 'Carlos...",119690785,0,Q125573387,2204877034,2024-07-15 10:55:11+00:00
2561,item,Q125863514,"{'en': {'language': 'en', 'value': 'Édgar Muri...","{'en': {'language': 'en', 'value': 'costarican...","{'en': [{'language': 'en', 'value': 'cholo'}],...","{'P31': [{'mainsnak': {'snaktype': 'value', 'p...","{'eswiki': {'site': 'eswiki', 'title': 'Édgar ...",119952960,0,Q125863514,2204901284,2024-07-15 11:29:07+00:00
2562,item,Q126888937,"{'en': {'language': 'en', 'value': 'Gabriela R...","{'en': {'language': 'en', 'value': 'United Nat...",{},"{'P31': [{'mainsnak': {'snaktype': 'value', 'p...",{},120897767,0,Q126888937,2186903399,2024-06-23 10:40:55+00:00
2563,item,Q128214388,"{'en': {'language': 'en', 'value': 'Leslie Fei...","{'en': {'language': 'en', 'value': 'Costa Rica...",{},"{'P31': [{'mainsnak': {'snaktype': 'value', 'p...","{'enwiki': {'site': 'enwiki', 'title': 'Leslie...",122196308,0,Q128214388,2231752127,2024-08-19 23:04:02+00:00


In [23]:
def get_triples_all_persons(persons_df):
    triples_all_persons = {}

    for i in range(len(persons_df)):
        person = persons_df.iloc[i]
        qid = person["id"]
        triples_per_person = get_triples_per_person(qid, person)
        triples_all_persons[qid] = triples_per_person
    return triples_all_persons

def get_triples_per_person(qid, person):
    triples_per_person = []

    for pid, values in person["claims"].items():
        triples = []
        value_types = []

        for snaks in values:
            value = None
            try:
                a = snaks["mainsnak"]["datavalue"]
            except KeyError:
                print(snaks["mainsnak"])
            if snaks["mainsnak"]["snaktype"] in ["novalue", "somevalue"]:
                continue
            else:
                value_type = snaks["mainsnak"]["datavalue"]["type"]
    
            if value_type not in value_types:
                value_types.append(value_type)
            if value_type == "string":
                value = snaks["mainsnak"]["datavalue"]["value"]
            if value_type == "time":
                value = snaks["mainsnak"]["datavalue"]["value"]["time"]
            if value_type == "wikibase-entityid":
                value = snaks["mainsnak"]["datavalue"]["value"]["id"]
            if value_type == "quantity":
                value = snaks["mainsnak"]["datavalue"]["value"]["amount"]
            if value is not None:
                triples.append((qid, pid, value))
        triples_per_person.append(triples)
    return list(chain(*triples_per_person))

def get_all_predicates(all_triples):
    return {triple[1] for triples in all_triples.values() for triple in triples}


def get_value_from_pid(pid, pid_to_label_map={}):
    if pid not in pid_to_label_map.keys():
        sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent="bench-gap")
        sparql.setQuery(f""" 
                        SELECT  *
                        WHERE
        {{
                        wd:{pid} rdfs:label ?label .
        }} 
        """)
        sparql.setReturnFormat(JSON)
        query_result = sparql.query().convert()
        query_result = get_query_result_in_en(query_result)
        try:
            value = query_result["label"]["value"]
            language = query_result["label"]["xml:lang"]
        except:
            print(pid, query_result)
            return None, pid_to_label_map
        pid_to_label_map[pid] = value
        return value, pid_to_label_map
    else:
        return pid_to_label_map[pid], pid_to_label_map
        
        
def get_query_result_in_en(query_result):
    is_english = False
    for result in query_result["results"]["bindings"]:
        if result["label"]["xml:lang"] == "en":
            is_english = True
        if is_english:
            return result
    if not is_english:
        raise ValueError("Non-english property")
    




In [9]:
Q123557115

NameError: name 'Q123557115' is not defined

In [26]:
all_triples = get_triples_all_persons(persons)
all_predicates = {triple[1] for triples in all_triples.values() for triple in triples}
pid_to_label_map = {}

{'snaktype': 'novalue', 'property': 'P54', 'datatype': 'wikibase-item'}
{'snaktype': 'somevalue', 'property': 'P570', 'datatype': 'time'}
{'snaktype': 'somevalue', 'property': 'P569', 'datatype': 'time'}
{'snaktype': 'somevalue', 'property': 'P570', 'datatype': 'time'}
{'snaktype': 'somevalue', 'property': 'P20', 'datatype': 'wikibase-item'}
{'snaktype': 'somevalue', 'property': 'P570', 'datatype': 'time'}
{'snaktype': 'somevalue', 'property': 'P570', 'datatype': 'time'}
{'snaktype': 'somevalue', 'property': 'P570', 'datatype': 'time'}
{'snaktype': 'novalue', 'property': 'P2446', 'datatype': 'external-id'}
{'snaktype': 'novalue', 'property': 'P2020', 'datatype': 'external-id'}


In [ ]:
for pid in all_predicates:
    _, pid_to_label_map = get_value_from_pid(pid, pid_to_label_map)

In [ ]:
pid_to_label_map